## Results Section 6.3: Custom Gates
In this notebook, we outline the post-processing procedure for Section 6.2. Specifically, we reproduce the results from Figure 3 and Table 5.

### Imports and setup

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from analyzer import main_analysis
import os

In [ ]:
source_file = "../../data/times.csv"
source_output = '../../data/output'
output_folder = "../../data/paper"
output_folder_qsyn = "../../qsyn/output"


# We manually define the Kang counts for the circuits, since parsing them automatically is very difficult
kang_counts = {
    'bit_measure': 0,
    'cluster': 13,
    'draper': 5,
    'flip': 6,
    'GHZ_by_iSWAP': 8,
    'GHZ_by_QFT': 7,
    'GHZ_from_100': 6,
    'GHZ_Game': 8,
    'indexed_bell': 4,
    'M_valued': 7,
    'QFT': 7,
    'teleportation': 4,
    'three_superpose': 3,
    'toffoli_by_sqrt_X': 5,
    'W_four': 7,
    'W_orthog': 8,
    'W_phased': 7,
    '1': 0,
    '2': 0,
    '3': 0,
    '4': 0,
    '5': 0,
}

### Figure 3

Loading the data

In [ ]:
# Synthetiq data
df = pd.read_csv(source_file, sep='\t', header=None)
df.columns = ['filename', 'time', 'std', 'n_runs', 'successful_runs', 'optimal_runs']
synthetiq = df.copy()
synthetiq = synthetiq[synthetiq["filename"].str.startswith("data/output/63/theirs/")]
synthetiq = synthetiq[np.logical_not(synthetiq["filename"].str.contains("count"))]
synthetiq['benchmark'] = synthetiq["filename"].apply(lambda x: x.split("/")[4])
should_add = synthetiq["optimal_runs"] < 100
synthetiq["time"] = synthetiq["time"] + should_add * (10800 / synthetiq["optimal_runs"] - synthetiq["time"])
synthetiq['num_gates'] = 0

for benchmark in synthetiq['filename']:
    benchmark_name = benchmark.replace("data/output/", "")
    min_num_gates = 100000000
    for file in os.listdir(os.path.join(source_output, benchmark_name)):
        # get the amount of non-empty lines in the file
        with open(os.path.join(source_output, benchmark_name, file)) as f:
            num_gates = sum(1 for line in f if line.strip()) - 3 # -3 because of the header
        min_num_gates = min(min_num_gates, num_gates)
    synthetiq.loc[synthetiq['filename'] == benchmark, 'num_gates'] = min_num_gates


In [ ]:
# loading the times for Kang and Oh
kang_data = []
for benchmark in synthetiq['benchmark']:
    if not os.path.exists(os.path.join(output_folder_qsyn, benchmark + '.txt')):
        continue
    with open(os.path.join(output_folder_qsyn, benchmark + '.txt')) as f:
        raw_output = f.read()
        failure = ''
        if 'Elapsed Time :' in raw_output:
            time = float(raw_output.split('Elapsed Time :')[1].split('\n')[0])
            num_gates = kang_counts[benchmark]
        elif 'TIMEOUT' in raw_output:
            failure = 'T/O'
            time = 24 * 3600
            num_gates = 0
        else:
            failure = 'N/A'
            time = 24 * 3600
            num_gates = 0
        kang_data.append({
            'benchmark': benchmark,
            'time': time,
            'num_gates': num_gates,
            'failure': failure
        })
kang = pd.DataFrame(kang_data)
kang

We order the benchmarks in an esthetically pleasing manner

In [ ]:
benchmarks = kang['benchmark'].unique()
# change the order of the benchmarks list to follow the order of (kang['num_gates'], synthetiq['num_gates'])
zero_to_high = lambda x: 1000 if x == 0 else x
benchmarks = sorted(benchmarks, key=lambda x: (zero_to_high(kang[kang['benchmark'] == x]['num_gates'].values[0]), 
                                                synthetiq[synthetiq['benchmark'] == x]['num_gates'].values[0]))
synthetiq = synthetiq.set_index('benchmark').loc[benchmarks].reset_index()
kang = kang.set_index('benchmark').loc[benchmarks].reset_index()

renames = {
        '1': 'CX',
        '2': 'sqrt_SWAP',
        '3': 'Permutation',
        '4': 'sqrt_iSWAP',
        '5': 'Oracle',
}
synthetiq['benchmark'] = synthetiq['benchmark'].apply(lambda x: renames.get(x, x))
kang['benchmark'] = kang['benchmark'].apply(lambda x: renames.get(x, x))

### Plot figure 3

In [ ]:
speedups = np.array(kang["time"]) / np.array(synthetiq["time"])
speedups = [speedup for i, speedup in enumerate(speedups) if kang["failure"][i] == '']
fig, ax = plt.subplots(figsize=(12, 5), ncols=2, width_ratios=[5, 1])
current_palette = sns.color_palette("Dark2", 2)
sns.boxplot(speedups, width=0.8, color=current_palette[0], ax=ax[1])
ax[1].set_yscale('log')
ax[1].set_xticklabels([])
ax[1].set_xticks([])
ax[1].set_title("Speedup")
# make the lines surrounding the plot disappear
sns.despine(left=True, bottom=True)
#make the background grey
ax[1].set_facecolor((0.95, 0.95, 0.95))
# make the title to the left
ax[1].title.set(x=0.05)
# background color very light grey
ax[0].set_facecolor((0.95, 0.95, 0.95))
ax[0].title.set(x=0)
ax[0].set_title("Gate Count")
sns.despine(left=True, bottom=True)

data = pd.DataFrame({'Source': ['Synthetiq' for i in range(len(synthetiq))] + ['Kang et al.' for i in range(len(kang))],
                    'count': np.concatenate([synthetiq['num_gates'], kang['num_gates']]), 
                    'label': np.concatenate([benchmarks, benchmarks])}
)

sns.set_palette(current_palette)
sns.barplot(data=data, y="count", x="label", hue="Source", edgecolor=(0.95, 0.95, 0.95), linewidth=0, ax=ax[0])

# rotate the xticks 45 degrees, making sure that they still appear under the barplots at the bottom
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=45, ha='right', fontsize=12)

ax[0].set_ylabel("")
ax[0].set_xlabel("")
ax[0].set_xlim(-0.5, 21.5)
ax[0].xaxis.labelpad = 20
ax[0].text(16.05, 2.7, "T/O", rotation=90, fontsize=11, color=current_palette[1], weight="bold")
ax[0].text(17.05, 4.2, "N/A", rotation=90, fontsize=11, color=current_palette[1], weight="bold")
ax[0].text(18.05, 4.2, "N/A", rotation=90, fontsize=11, color=current_palette[1], weight="bold")
ax[0].text(19.05, 6.2, "T/O", rotation=90, fontsize=11, color=current_palette[1], weight="bold")
ax[0].text(20.05, 6.7, "N/A", rotation=90, fontsize=11, color=current_palette[1], weight="bold")
ax[0].text(21.05, 9.2, "T/O", rotation=90, fontsize=11, color=current_palette[1], weight="bold")
# set title font size
ax[0].title.set_fontsize(16)
ax[0].tick_params(axis='y', labelsize=16)
# set the legend font size
ax[0].legend(fontsize=16)
ax[1].title.set_fontsize(16)

# set xticks font size
ax[1].tick_params(axis='y', labelsize=16)

plt.tight_layout() 

fig.savefig('../../data/paper/fig_3.pdf', bbox_inches='tight')

### Table 5

In [ ]:
results = []
os.makedirs(os.path.join(output_folder, '63', 'stackexchange'), exist_ok=True)
for folder in os.listdir(os.path.join(source_output, '63', 'stackexchange')):
    output = dict()
    full_folder = os.path.join(source_output, '63', 'stackexchange', folder)
    # search in df for the time
    element = df[df['filename'] == full_folder[6:] + '/'].to_dict('records')[0]
    output['name'] = {
        '1': 'CX',
        '2': '\sqrt{SWAP}',
        '3': 'Permutation',
        '4': '\sqrt{iSWAP}',
        '5': 'Oracle',
    }.get(folder, 'None')
    output['time'] = element['time']
    _, _, _, circ, _, _ = main_analysis(full_folder)
    output['num_qubits'] = circ.circuit.num_qubits
    output['expensive_count'] = circ.count
    circ.circuit.qasm(formatted=True, filename=os.path.join(output_folder, '63', 'stackexchange', f'{folder}.qasm'))
    results.append(output)

output = pd.DataFrame(results)
output.to_csv(os.path.join(output_folder, '63', 'table5.csv'), index=False)